## Zadania: LangGraph - wzorce agentowe


Zadanie \
Agent Supervisor
1. Zmień pytanie przypisane do zmiennej topic i uruchom ponownie kod.
2. Zmień prompty dla węzłów "researcher" oraz "expert" tak aby pelnili inne role, np. "scenarzysta" i "scenograf" i uruchom ponownie kod.

In [1]:
import operator
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class SupervisorState(TypedDict):
    """State for supervisor pattern with multiple agents."""
    topic: str
    messages: Annotated[List[str], operator.add]
    next_agent: str
    final_answer: str


def researcher_agent(state: SupervisorState) -> dict:
    """Researcher agent gathers information about the topic."""
    sys = ( # Zmodyfikuj prompt poniżej
        "You are a researcher. Your job is to gather key facts and information "
        "about the given topic. Provide 2-3 key points. Be concise."
    )
    messages_for_llm = [
        ("system", sys),
        ("user", f"Research this topic: {state['topic']}")
    ]
    resp = llm.invoke(messages_for_llm)
    research_msg = f"RESEARCHER: {resp.content}"
    return {"messages": [research_msg]}


def expert_agent(state: SupervisorState) -> dict:
    """Expert agent analyzes and provides insights based on research."""
    sys = ( # Zmodyfikuj prompt poniżej
        "You are an expert analyst. Review the research provided and give "
        "your expert analysis and conclusions. Be specific and insightful."
    )
    # Get context from previous messages
    context = "\n".join(state["messages"])
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nPrevious research:\n{context}\n\nProvide your expert analysis.")
    ]
    resp = llm.invoke(messages_for_llm)
    expert_msg = f"EXPERT: {resp.content}"
    return {"messages": [expert_msg]}


def supervisor_agent(state: SupervisorState) -> dict:
    """Supervisor decides which agent should act next or if discussion should end."""
    sys = (
        "You are a supervisor managing a research discussion between a RESEARCHER and an EXPERT. "
        "Based on the conversation so far, decide what should happen next:\n"
        "- Return 'researcher' if we need initial research or more information\n"
        "- Return 'expert' if research is done and we need expert analysis\n"
        "- Return 'end' if both research and expert analysis are complete\n\n"
        "Respond with ONLY one word: researcher, expert, or end"
    )

    context = "\n".join(state["messages"]) if state["messages"] else "No discussion yet"
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nConversation:\n{context}\n\nWhat's next?")
    ]
    resp = llm.invoke(messages_for_llm)
    next_step = resp.content.strip().lower()

    # Ensure valid response
    if next_step not in ["researcher", "expert", "end"]:
        next_step = "end"

    return {"next_agent": next_step}


def finalize_answer(state: SupervisorState) -> dict:
    """Compile final answer from the discussion."""
    sys = (
        "Summarize the research discussion into a clear, concise final answer. "
        "Include key findings and expert insights."
    )
    context = "\n".join(state["messages"])
    messages_for_llm = [
        ("system", sys),
        ("user", f"Topic: {state['topic']}\n\nDiscussion:\n{context}\n\nProvide final summary:")
    ]
    resp = llm.invoke(messages_for_llm)
    return {"final_answer": resp.content}


def route_supervisor(state: SupervisorState) -> str:
    """Route based on supervisor's decision."""
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "end":
        return "finalize"
    return next_agent

supervisor_graph = StateGraph(SupervisorState)

supervisor_graph.add_node("supervisor", supervisor_agent)
supervisor_graph.add_node("researcher", researcher_agent)
supervisor_graph.add_node("expert", expert_agent)
supervisor_graph.add_node("finalize", finalize_answer)

supervisor_graph.add_edge(START, "supervisor")

supervisor_graph.add_conditional_edges(
    "supervisor",
    route_supervisor,
    {
        "researcher": "researcher",
        "expert": "expert",
        "finalize": "finalize"
    }
)

supervisor_graph.add_edge("researcher", "supervisor")
supervisor_graph.add_edge("expert", "supervisor")

supervisor_graph.add_edge("finalize", END)

supervisor_agent_graph = supervisor_graph.compile(debug=True)

topic = "What are the main benefits of using LangGraph for building AI agents?" # <- zmodyfikuj zadane pytanie

initial_state = {
    "topic": topic,
    "messages": [],
    "next_agent": "",
    "final_answer": ""
}

result = supervisor_agent_graph.invoke(initial_state)

print(f"TOPIC: {topic}\n")
print("=" * 80)
print("\nDISCUSSION:")
print("-" * 80)
for msg in result["messages"]:
    print(f"\n{msg}\n")
print("=" * 80)
print(f"\nFINAL ANSWER:\n{result['final_answer']}")

[values] {'topic': 'What are the main benefits of using LangGraph for building AI agents?', 'messages': [], 'next_agent': '', 'final_answer': ''}
[updates] {'supervisor': {'next_agent': 'researcher'}}
[values] {'topic': 'What are the main benefits of using LangGraph for building AI agents?', 'messages': [], 'next_agent': 'researcher', 'final_answer': ''}
[updates] {'researcher': {'messages': ['RESEARCHER: 1. **Modular Design**: LangGraph offers a modular architecture that allows developers to easily integrate various components and functionalities, facilitating the rapid development and deployment of AI agents tailored to specific tasks.\n\n2. **Enhanced Natural Language Processing**: The platform leverages advanced natural language processing capabilities, enabling AI agents to understand and generate human-like text, which improves user interaction and engagement.\n\n3. **Scalability and Flexibility**: LangGraph is designed to scale efficiently, accommodating a wide range of applicat